In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_dataset

import numpy as np

from sklearn import metrics

import joblib

2024-02-26 01:08:18.185665: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-26 01:08:18.188237: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 01:08:18.216623: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 01:08:18.216671: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 01:08:18.217583: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [3]:
def calculate_performance_metrics(x_test, y_test, model):

    # Predictions
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    report = metrics.classification_report(y_test, y_pred_classes, output_dict=True, zero_division=1)
    
    precision_macro = report['macro avg']['precision']
    precision_weighted = report['weighted avg']['precision']
    accuracy = report['accuracy']
    recall_macro = report['macro avg']['recall']
    recall_weighted = report['weighted avg']['recall']
    f1_macro = report['macro avg']['f1-score']
    f1_weighted = report['weighted avg']['f1-score']
    auc = metrics.roc_auc_score(y_test, y_pred, multi_class='ovr')

    print(f"Accuracy: {accuracy}\nAUC: {auc}")
    
    print("\nmacro")
    print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")

    print("\nweighted")
    print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
    print()
    
    # Confusion matrix for FNR, TNR, FPR, TPR
    cm = metrics.confusion_matrix(y_test, y_pred_classes)
    def calculate_rates(conf_matrix, class_index):
        tp = conf_matrix[class_index, class_index]
        fn = np.sum(conf_matrix[class_index, :]) - tp
        fp = np.sum(conf_matrix[:, class_index]) - tp
        tn = np.sum(conf_matrix) - (tp + fn + fp)
    
        fnr = fn / (fn + tp)
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        tpr = tp / (tp + fn)
        return fnr, tnr, fpr, tpr

    # Calculate and aggregate rates
    fnrs, tnrs, fprs, tprs = [], [], [], []
    for i in range(cm.shape[0]):
        fnr, tnr, fpr, tpr = calculate_rates(cm, i)
        fnrs.append(fnr)
        tnrs.append(tnr)
        fprs.append(fpr)
        tprs.append(tpr)
    
    mean_fnr = np.mean(fnrs)
    mean_tnr = np.mean(tnrs)
    mean_fpr = np.mean(fprs)
    mean_tpr = np.mean(tprs)

    # Printing the mean metrics
    print(f"Mean FNR: {mean_fnr}\nMean TNR: {mean_tnr}\nMean FPR: {mean_fpr}\nMean TPR: {mean_tpr}")

In [4]:
x_test = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/x_test.npy")
x_train = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/x_train.npy")
x_val = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/x_val.npy")
y_test = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/y_test.npy")
y_train = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/y_train.npy")
y_val = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/y_val.npy")

In [5]:
model = joblib.load("/home/jovyan/X-IIoT/dl/dl_new/model/dnn.joblib")

2024-02-26 01:08:22.402803: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://adec3952bac74758b3d2fa329f3012f8: INVALID_ARGUMENT: ram://adec3952bac74758b3d2fa329f3012f8 is a directory.


In [6]:
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=19,
    input_shape=(58,),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy(),
)

In [7]:
attack = FastGradientMethod(estimator=classifier, eps=0.01)

x_test_adv = attack.generate(x=x_test)  # x_test is your MNIST test images

In [8]:
calculate_performance_metrics(x_test_adv, y_test, model)

4383/4383 [==============================] - 3s 578us/step
Accuracy: 0.9314412037367182
AUC: 0.9957207294148642

macro
Precision: 0.8387937084154355
Recall: 0.7174022815363879
F1 Score: 0.7433135682648767

weighted
Precision: 0.9434803058559201
Recall: 0.9314412037367182
F1 Score: 0.9315831201536421

Mean FNR: 0.28259771846361226
Mean TNR: 0.9948226535788895
Mean FPR: 0.0051773464211104635
Mean TPR: 0.7174022815363879


In [9]:
attack = FastGradientMethod(estimator=classifier, eps=0.05)

x_test_adv = attack.generate(x=x_test)  # x_test is your MNIST test images

In [10]:
calculate_performance_metrics(x_test_adv, y_test, model)

4383/4383 [==============================] - 3s 598us/step
Accuracy: 0.6271696498609427
AUC: 0.943599114051786

macro
Precision: 0.4110921179647813
Recall: 0.3334809939575236
F1 Score: 0.33029031048270224

weighted
Precision: 0.7442797695863355
Recall: 0.6271696498609427
F1 Score: 0.6565589169653229

Mean FNR: 0.6665190060424763
Mean TNR: 0.9732789075476335
Mean FPR: 0.026721092452366604
Mean TPR: 0.3334809939575236


In [6]:
attack = FastGradientMethod(estimator=classifier, eps=0.1)

x_test_adv = attack.generate(x=x_test)  # x_test is your MNIST test images

In [7]:
calculate_performance_metrics(x_test_adv, y_test, model)

4383/4383 [==============================] - 3s 596us/step
Accuracy: 0.49437352920202526
AUC: 0.8176632845118779

macro
Precision: 0.16567155187903126
Recall: 0.13082570933740958
F1 Score: 0.12326745259155923

weighted
Precision: 0.538200876919635
Recall: 0.49437352920202526
F1 Score: 0.5006809222699486

Mean FNR: 0.8691742906625904
Mean TNR: 0.9600432784189556
Mean FPR: 0.03995672158104433
Mean TPR: 0.13082570933740958


In [8]:
attack = FastGradientMethod(estimator=classifier, eps=0.15)

x_test_adv = attack.generate(x=x_test)  # x_test is your MNIST test images

In [9]:
calculate_performance_metrics(x_test_adv, y_test, model)

4383/4383 [==============================] - 3s 593us/step
Accuracy: 0.4402053768808386
AUC: 0.7423987187170208

macro
Precision: 0.11165791827761024
Recall: 0.08551182243364797
F1 Score: 0.08050308239714096

weighted
Precision: 0.4563365720504677
Recall: 0.4402053768808386
F1 Score: 0.4340487647754112

Mean FNR: 0.9144881775663519
Mean TNR: 0.9531055453033366
Mean FPR: 0.04689445469666337
Mean TPR: 0.08551182243364797


In [10]:
attack = FastGradientMethod(estimator=classifier, eps=0.2)

x_test_adv = attack.generate(x=x_test)  # x_test is your MNIST test images

In [11]:
calculate_performance_metrics(x_test_adv, y_test, model)

4383/4383 [==============================] - 3s 584us/step
Accuracy: 0.43582685587962633
AUC: 0.6930287084002807

macro
Precision: 0.08759920092225167
Recall: 0.07398625679397974
F1 Score: 0.06894724987237995

weighted
Precision: 0.41679799119834054
Recall: 0.43582685587962633
F1 Score: 0.41647092512606265

Mean FNR: 0.9260137432060203
Mean TNR: 0.9510961370507214
Mean FPR: 0.048903862949278606
Mean TPR: 0.07398625679397974


In [12]:
attack = FastGradientMethod(estimator=classifier, eps=0.25)

x_test_adv = attack.generate(x=x_test)  # x_test is your MNIST test images

In [13]:
calculate_performance_metrics(x_test_adv, y_test, model)

4383/4383 [==============================] - 3s 581us/step
Accuracy: 0.4383940668901091
AUC: 0.6602117493404339

macro
Precision: 0.07715481761208043
Recall: 0.0679744016802982
F1 Score: 0.06454649886450971

weighted
Precision: 0.40261411685557036
Recall: 0.4383940668901091
F1 Score: 0.41252537124031846

Mean FNR: 0.9320255983197017
Mean TNR: 0.9504375500218522
Mean FPR: 0.04956244997814759
Mean TPR: 0.0679744016802982


In [14]:
attack = FastGradientMethod(estimator=classifier, eps=0.3)

x_test_adv = attack.generate(x=x_test)  # x_test is your MNIST test images

In [15]:
calculate_performance_metrics(x_test_adv, y_test, model)

4383/4383 [==============================] - 3s 582us/step
Accuracy: 0.4335876773871497
AUC: 0.6341835832874074

macro
Precision: 0.06776934628984983
Recall: 0.0622013986882243
F1 Score: 0.059394657823542404

weighted
Precision: 0.3893738980285425
Recall: 0.4335876773871497
F1 Score: 0.40463299469211134

Mean FNR: 0.9377986013117757
Mean TNR: 0.9495033137291368
Mean FPR: 0.05049668627086317
Mean TPR: 0.0622013986882243
